# Consulting Project

## Nhập dữ liệu

In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
import findspark
findspark.init()

In [2]:
sc= SparkContext(appName= 'Chapter7-Excersice2', master= 'local')
ss= SparkSession(sc)

In [3]:
path= '/Users/vovanthuong/Desktop/9 - Big Data in Machine Learning/Data/Chapter7/customer_churn.csv'
df= ss.read.csv(path, inferSchema= True, header= True)

In [6]:
df.show(2)

+----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------+-----+
|           Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|   Company|Churn|
+----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------+-----+
|Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|Harvey LLC|    1|
|   Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|Wilson PLC|    1|
+----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------+-----+
only showing top 2 rows



## Kiểm tra tổng quát bộ dữ liệu

In [7]:
df.count()

900

In [32]:
# Kiểm tra kiểu dữ liệu và số lượng các giá trị để lựa chọn đưa vào phân tích
for col in df.columns:
    dtype= df.select(col).dtypes[0][1]
    count= df.select(col).distinct().count()
    print(col + ': ' + str(count) + ' - ' + dtype)

Names: 899 - string
Age: 36 - double
Total_Purchase: 900 - double
Account_Manager: 2 - int
Years: 418 - double
Num_Sites: 12 - double
Onboard_date: 900 - timestamp
Location: 900 - string
Company: 873 - string
Churn: 2 - int


In [34]:
# Các cột được lựa chọn vào phân tích:
column_selected= df.columns
column_selected.remove('Names')
column_selected.remove('Onboard_date')
column_selected.remove('Location')
column_selected.remove('Company')
column_selected

['Age', 'Total_Purchase', 'Account_Manager', 'Years', 'Num_Sites', 'Churn']

In [35]:
# Xem mô tả tổng quát
df.select(column_selected).describe().toPandas()

,summary,Age,Total_Purchase,Account_Manager,Years,Num_Sites,Churn
0,count,900,900,900,900,900,900
1,mean,41.81666666666667,10062.82403333334,0.4811111111111111,5.27315555555555,8.587777777777777,0.16666666666666666
2,stddev,6.127560416916251,2408.644531858096,0.4999208935073339,1.274449013194616,1.7648355920350969,0.3728852122772358
3,min,22.0,100.0,0,1.0,3.0,0
4,max,65.0,18026.01,1,9.15,14.0,1


In [40]:
# Kiểm tra giá trị NaN
from pyspark.sql.functions import count, when, isnan, isnull, col
nan_data= df.select([count(when(isnan(c), c)).alias(c + '_nan') for c in column_selected]).toPandas().T
nan_data

,0
Age_nan,0
Total_Purchase_nan,0
Account_Manager_nan,0
Years_nan,0
Num_Sites_nan,0
Churn_nan,0


In [41]:
# Kiểm tra giá trị Null
null_data= df.select([count(when(isnull(c), c)).alias(c + '_null') for c in column_selected]).toPandas().T
null_data

,0
Age_null,0
Total_Purchase_null,0
Account_Manager_null,0
Years_null,0
Num_Sites_null,0
Churn_null,0


## Xác định input và output

In [58]:
output_column= 'Churn'
input_columns= column_selected.copy()
input_columns.remove(output_column)

## Chuẩn bị dữ liệu train và dữ liệu test

In [42]:
train, test= df.randomSplit([0.8, 0.2])

## Xây dựng pipeline để làm sạch và chuẩn hóa dữ liệu

In [44]:
# Kiểm tra kiểu dữ liệu và số lượng các giá trị để lựa chọn đưa vào phân tích
for col in column_selected:
    dtype= df.select(col).dtypes[0][1]
    count= df.select(col).distinct().count()
    print(col + ': ' + str(count) + ' - ' + dtype)

Age: 36 - double
Total_Purchase: 900 - double
Account_Manager: 2 - int
Years: 418 - double
Num_Sites: 12 - double
Churn: 2 - int


In [61]:
from pyspark.ml.feature import SQLTransformer, VectorAssembler, MinMaxScaler
from pyspark.ml import Pipeline

In [62]:
select_columns= SQLTransformer(statement= 'SELECT {cols} FROM __THIS__'.format(cols= ','.join(column_selected)))

In [63]:
vec_assembler= VectorAssembler(outputCol= 'features', inputCols= input_columns)

In [64]:
mm_scaler= MinMaxScaler(inputCol= 'features', outputCol= 'features_scaled')

In [68]:
pipe_process= Pipeline(stages=[select_columns, vec_assembler, mm_scaler])
process= pipe_prcess.fit(train)

In [70]:
train_cleaned= process.transform(train)
train_cleaned.show(5)

+----+--------------+---------------+-----+---------+-----+--------------------+--------------------+
| Age|Total_Purchase|Account_Manager|Years|Num_Sites|Churn|            features|     features_scaled|
+----+--------------+---------------+-----+---------+-----+--------------------+--------------------+
|45.0|       9598.03|              0|  5.0|      7.0|    0|[45.0,9598.03,0.0...|[0.53488372093023...|
|55.0|      10056.55|              0| 4.98|      8.0|    0|[55.0,10056.55,0....|[0.76744186046511...|
|55.0|       8243.28|              0| 3.54|      6.0|    0|[55.0,8243.28,0.0...|[0.76744186046511...|
|36.0|      12309.23|              1| 4.76|      9.0|    0|[36.0,12309.23,1....|[0.32558139534883...|
|48.0|       6495.01|              1| 5.57|     12.0|    1|[48.0,6495.01,1.0...|[0.60465116279069...|
+----+--------------+---------------+-----+---------+-----+--------------------+--------------------+
only showing top 5 rows



## Xây dựng mô hình Logistic

In [56]:
from pyspark.ml.classification import LogisticRegression
lgr= LogisticRegression(featuresCol= 'features_scaled', labelCol= output_column, predictionCol= output_column + '_predict')

In [71]:
lgr_model= lgr.fit(train_cleaned)

## Đánh giá mô hình

### Trên tập train

In [72]:
train_evaluate= lgr_model.evaluate(train_cleaned)

In [73]:
train_evaluate.accuracy

0.9010840108401084

In [74]:
train_evaluate.areaUnderROC

0.911732434397514

In [77]:
train_evaluate.precisionByLabel

[0.9221183800623053, 0.7604166666666666]

In [78]:
train_evaluate.recallByLabel

[0.9626016260162602, 0.5934959349593496]

In [80]:
df.select(output_column).groupBy(output_column).count().show()

+-----+-----+
|Churn|count|
+-----+-----+
|    1|  150|
|    0|  750|
+-----+-----+



### Trên tập test

In [85]:
test_cleaned= process.transform(test)

In [86]:
test_evaluate= lgr_model.evaluate(test_cleaned)

In [87]:
test_evaluate.accuracy

0.8827160493827161

In [88]:
test_evaluate.areaUnderROC

0.8888888888888895

In [93]:
test_evaluate.precisionByLabel

[0.8972602739726028, 0.75]

In [94]:
test_evaluate.recallByLabel

[0.9703703703703703, 0.4444444444444444]

## Dự đoán trên tập dữ liệu mới

In [95]:
path_df_new= '/Users/vovanthuong/Desktop/9 - Big Data in Machine Learning/Data/Chapter7/new_customers.csv'
df_new= ss.read.csv(path_df_new, inferSchema= True, header= True)

In [107]:
select_inputs= SQLTransformer(statement= 'SELECT Names, {cols} FROM __THIS__'.format(cols= ','.join(input_columns)))
pipe_process_dfnew= Pipeline(stages=[select_inputs, vec_assembler, mm_scaler])
process_dfnew= pipe_process_dfnew.fit(train)

In [108]:
df_new_cleaned= process_dfnew.transform(df_new)

In [109]:
df_new_result= lgr_model.transform(df_new_cleaned)

In [112]:
df_new_result.select('Names', 'Churn_predict').show(20)

+--------------+-------------+
|         Names|Churn_predict|
+--------------+-------------+
| Andrew Mccall|          0.0|
|Michele Wright|          1.0|
|  Jeremy Chang|          1.0|
|Megan Ferguson|          1.0|
|  Taylor Young|          0.0|
| Jessica Drake|          1.0|
+--------------+-------------+

